# Preprocessing Student Answer
Create scoring web apps and validate the scoring result.

Install Linux tools and only required for the first run.

In [ ]:
!sudo apt-get update
!sudo apt-get install ffmpeg libsm6 libxext6 -y

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
project_id = 'cyrus-testing-2023'
!gcloud config set project {project_id}
!gcloud auth application-default set-quota-project {project_id}

In [ ]:
from google.cloud import aiplatform
import vertexai

aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project=project_id
)

vertexai.init(project=project_id, location="us-central1")

In [ ]:
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

def call_llm(prompt=None, max_output_tokens=1024, temperature=0, top_p=0.3):
    if prompt is None:
        raise ValueError("Prompt cannot be None")
    
    model = GenerativeModel("gemini-1.0-pro-001")
    generation_config = {
        "max_output_tokens": max_output_tokens,
        "temperature": temperature,
        "top_p": top_p,
    }
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
    
    retry = 0
    while retry < 3:
        try:
            responses = model.generate_content(
                [prompt],
                generation_config=generation_config,
                safety_settings=safety_settings,
                stream=True,
            )
            text = ""
            for response in responses:
                text += response.text
            return text
        except Exception as e:
            print(e)
            retry += 1        
    return ""


def call_llm_visual(prompt:str, filePath:str):    
    model = GenerativeModel("gemini-1.0-pro-vision-001")    
    with open(filePath, "rb") as f:
        data = f.read()
    image1 = Part.from_data(mime_type="image/png", data=data)
    generation_config = {
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 0.5,
    }
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
    responses = model.generate_content(
        [image1, prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )
    text = ""
    for response in responses:        
        text += response.text
    return text

In [ ]:
p = """
Extract question and answer from the image.
Each Cell contains a question and answer.
Question is the printed text in left upper coner of the cell 
Question is informat 1), 2), 3)...
Asnwer is in handwrting text after the question's ')'.

Special handling 3 optional fields in the top of image, and they are not in cell.
1. Name: <- Question is "Name", answer is hand writing text.
2. Student ID: <- Question is "StudentID", answer is hand writing text.
3. Class: <- Question is "Class", answer is hand writing text.

Return JSON arrray object with question and answer.
{
    'question': '1)',
    'answer': 'HAND WRITING TEXT'
}
Output:
"""
a = call_llm_visual(p, "/workspaces/ai-scoring-handwriting-assignment/marking_form/TestScript/images/0.jpg")
print(a)

In [ ]:
pdf_file = "../data/TestScript.pdf"
standard_answer = pdf_file.replace(".pdf", ".xlsx")
question_with_answer_docx= pdf_file.replace("TestScript.pdf", "QuestionAndAsnwer.docx")

In [ ]:
import os

file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]
base_path = "../marking_form/" + file_name
base_path_images = base_path + "/images/"
base_path_annotations = base_path+"/annotations/"
base_path_questions = base_path+"/questions"
base_path_javascript = base_path+"/javascript"
extracted_questions_excel = base_path + "/extracted_questions.xlsx"
extracted_questions_review_excel = base_path + "/extracted_questions_review.xlsx"

# create directory tree for base_path_images
os.makedirs(base_path_images, exist_ok=True)
os.makedirs(base_path_annotations, exist_ok=True)
os.makedirs(base_path_questions, exist_ok=True)
os.makedirs(base_path_javascript, exist_ok=True)

In [ ]:
import json
annotations_path = base_path_annotations + "annotations.json"
with open(annotations_path, "r") as f: 
    annotations = json.load(f)          

#flatten annotations to list 
annotations_list = []
for page in annotations:
    for annotation in annotations[page]:
        annotation["page"] = int(page)
        # x to left, y to top
        annotation["left"] = annotation["x"]
        annotation["top"] = annotation["y"]
        annotation.pop("x")
        annotation.pop("y")
        annotations_list.append(annotation) 
annotations_list

# convert annotations_list to dict with key with label
annotations_dict = {}
for annotation in annotations_list:
    annotations_dict[annotation["label"]] = annotation
# annotations_dict


In [ ]:
# extract list of label from annotations as questions
questions = []
for annotation in annotations_list:
    if annotation["label"] not in questions:
        questions.append(annotation["label"])
# remove 'NAME', 'ID', 'CLASS' if exists in questions
if 'NAME' in questions:
    questions.remove('NAME')
if 'ID' in questions:
    questions.remove('ID')
if 'CLASS' in questions:
    questions.remove('CLASS')    

# sort questions 
questions.sort()
# question_with_answer = questions.copy()
questions = ['NAME', 'ID', 'CLASS'] + questions
questions

## Extract Question, Answer and mark.

In [ ]:
import mammoth

def convert_image(image):  
    return {
        "src": ""
    }

with open("/workspaces/ai-scoring-handwriting-assignment/data/Sample.docx", "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file, convert_image=mammoth.images.img_element(convert_image))
    html  = result.value # The generated HTML
    messages = result.messages # Any messages, such as warnings during conversion
html 

In [ ]:
def get_question_answer(raw_text,questions):
    prompt = f"""
    Example 1 - single-level question HTML structure:
    ++++++++++++++++++++++++++
    Session2.2: The following questions Governance and Compliance.    
    18)	How can an Azure region with 1 available zone to ensure high availability? [1 mark]
    regions are paired
    ++++++++++++++++++++++++++

    Example 1 - JSON structure output:
    ++++++++++++++++++++++++++
    [
        {{
            "questionId":"18", 
            "parent": null,
            "question":"How can an Azure region with 1 available zone to ensure high availability?",
            "answer":"regions are paired",
            "mark": 1,
        }}
    ]
    ++++++++++++++++++++++++++

    
    Example 2 - mult-level question HTML structure:
    ++++++++++++++++++++++++++  
    Session2.2: The following questions Governance and Compliance.    
        19)	With pandas data frame (df), write down the program code for the following cases:  
            a.	To preview first 5 rows data. [1 mark]
            df.head(5)
    ++++++++++++++++++++++++++ 
    Example 2 - mult-level JSON structure output:
    ++++++++++++++++++++++++++
    [     
        {{
            "questionId":"19a",
            "parent":"With pandas data frame (df), write down the program code for the following cases: "
            "question":"To preview first 5 rows data.",
            "answer":"df.head(5)",
            "mark": 1,
        }}
    ]
    ++++++++++++++++++++++++++

    1. questionId can be mult-level e.g. 19a, 19b, ...etc.
    2. mult-level question includes parent question. 
    3. The questionId should be in the following list.
        {questions}

    The content of the HTML file is as follows:
    <INPUT>
    
    {raw_text}

    </INPUT>    

    IMPORTANT: 
    1. The output must be a JSON array of object contains only 'questionId', 'parent', 'question', 'answer', and 'mark'!
    2. no duplicate attribute.
    3. Escaped unescaped character in value string!
    4. Don't output ``` at the end of the JSON array.
    OUTPUT:
    """
    return call_llm(prompt,max_output_tokens=8192)
questions_string = ', '.join(questions)
raw_result = get_question_answer(html,questions_string)
print(raw_result)


It always return ``` after the end of JSON array!

In [ ]:
last_occurrence = raw_result.rfind(']')
if last_occurrence != -1:
    result = raw_result[:last_occurrence + 1]
print(result)

In [ ]:
json.loads(result)

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Question(BaseModel):
    questionId: str
    parent: Optional[str]
    question: str
    answer: str
    mark: int

errors_questions = []
extracted_questions = []
for item in json.loads(result):
    try:
        q = Question(**item)
        extracted_questions.append(q)
    except Exception as e: 
        errors_questions.append(item)

In [ ]:
import pandas as pd

# Convert extracted_questions list to DataFrame
extracted_questions_df = pd.DataFrame([vars(s) for s in extracted_questions])
errors_questions_df = pd.DataFrame(errors_questions)

# Create a Pandas Excel writer using the file name
with pd.ExcelWriter(extracted_questions_excel) as writer:
    # Write the errors_questions_df DataFrame to a sheet named 'Errors'
    errors_questions_df.to_excel(writer, sheet_name='Errors', index=False)    
    # Write the extracted_questions_df DataFrame to a sheet named 'Extracted Questions'
    extracted_questions_df.to_excel(writer, sheet_name='Extracted Questions', index=False)


### You need to Review the file fix the incorrect data manually.
Rename extracted_questions_excel.xlsx to extracted_questions_review_excel.xlsx.
Load back the excel.

In [ ]:
import pandas as pd
# Load the DataFrame from the "Extracted Questions" sheet
standard_answer_df = pd.read_excel(extracted_questions_review_excel, sheet_name='Extracted Questions')

# Print the loaded DataFrame
standard_answer_df.head()


## Get Student Name List.

In [ ]:
## load standard_answer to dataframe
import pandas as pd
name_list_df = pd.read_excel(standard_answer, sheet_name="NameList")
name_list_df.head()

### Check any unmatch between Annotation and answer from Gemini Extraction

In [ ]:
from termcolor import colored

for question in questions:
    if question not in standard_answer_df["questionId"].values:
        print(colored("Question {} is not in standard_answer!".format(question), 'red'))

for question in standard_answer_df["questionId"].values:
    if question not in questions:
        print(colored("Question {} is not in annotations!".format(question), 'red'))
            

In [ ]:
standard_answer = standard_answer_df.set_index("Question").to_dict()["Answer"]
standard_answer

In [ ]:
standard_mark = standard_answer_df.set_index("Question").to_dict()["Mark"]
standard_mark

Check for the regeneration of question.

In [ ]:
import os
import json

questionAndControl = {}
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file == "control.json":
            question = path[len(base_path_questions) + 1 :]
            f = open(os.path.join(path, file))
            data = json.load(f)
            if "regenerate" in data:
                questionAndControl[question] = data
            f.close()

questionAndControl

In [ ]:
from distutils.dir_util import copy_tree
import shutil
import os

from_directory = os.path.join(os.getcwd(), "..","templates", "javascript")
copy_tree(from_directory, base_path_javascript)
ico = os.path.join(os.getcwd(), "..","templates", "favicon.ico")
# copy ico file  to base_path
shutil.copyfile(ico, base_path+"/favicon.ico")

Generate the index.html

In [ ]:
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

file_loader = FileSystemLoader("../templates")
env = Environment(loader=file_loader)
template = env.get_template("index.html")

output = template.render(
    studentsScriptFileName=file_name,
    textAnswer=questions,
    optionAnswer=[],
)
# open text file
path = Path(os.path.join(base_path, "index.html"))
text_file = open(path, "w")
text_file.write(output)
text_file.close()

In [ ]:
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

def ocr(prompt:str, filePath:str):    
    model = GenerativeModel("gemini-1.0-pro-vision-001")    
    with open(filePath, "rb") as f:
        data = f.read()
    image1 = Part.from_data(mime_type="image/png", data=data)
    generation_config = {
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 0.5,
    }
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
    responses = model.generate_content(
        [image1, prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )
    text = ""
    for response in responses:        
        text += response.text
    return text

In [ ]:
import tempfile
from PIL import Image, ImageEnhance

def ocr_image_from_file(question, image_path, left, top, width, height):
    if question == "NAME" :
        return ""
    
    imageFile = tempfile.NamedTemporaryFile(suffix=".png").name
    with Image.open(image_path) as im:
        # The crop method from the Image module takes four coordinates as input.
        # The right can also be represented as (left+width)
        # and lower can be represented as (upper+height).
        (left, top, right, lower) = (
            left,
            top,
            left + width,
            top + height,
        )
        # Here the image "im" is cropped and assigned to new variable im_crop
        im_crop = im.crop((left, top, right, lower))
        imageEnhance = ImageEnhance.Sharpness(im_crop)
        # showing resultant image
        im_crop = imageEnhance.enhance(3)
        im_crop.save(imageFile, format="png")
        
    if question == "ID" :
        text_message = """
            Extract text in this image.
            It is a Student ID in 9 digit number.
            Answer just extracted Student ID and don't answer anything else.
            If you cannot extract Student ID, please return 'No text found!!!'.
            """
    else:    
        text_message ="""
            Extract text in this image in English and number.
            Answer just extracted text and don't answer anything else.
            If you cannot extract text, please return 'No text found!!!'."""       

    try:
        ocr_text = ocr(text_message, imageFile)  
        print(question, image_path ,ocr_text) 
        if ocr_text == "No text found!!!":
            return ""
        return ocr_text
    except Exception as e:
        print(question, image_path, e)    
        return ""

### Change Student Answer


In [ ]:
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def similarity_score(standard_answer, submitted_answer):
    model = GenerativeModel("gemini-1.0-pro-001")
    generation_config = {
        "max_output_tokens": 1024,
        "temperature": 0,
        "top_p": 0.3,
    }
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
    prompt = f'''
You are a grader to give a single digit floating point score from 0 to 1.
Give the score according to the similarity of the meaning between the standard answer and submitted answer.
Only reply a floating point number.
Standard answer:

{standard_answer}

Submitted Answer:

{submitted_answer}

Score: 
'''
    retry = 0
    while retry < 3:
        responses = model.generate_content(
            [prompt],
            generation_config=generation_config,
            safety_settings=safety_settings,
            stream=True,
        )
        text = ""
        for response in responses:
            text += response.text
        try:
            score = float(text)
            return score
            break  # If the conversion is successful, break the loop
        except ValueError:
            print("Retry")
            retry += 1
            continue
    return 0
        

def calculate_similarity(answers, question):
    # Add the standard answer to the head of list.
    if question not in standard_answer:
        ## return list of 0 in len of answers
        return [0] * len(answers)
    answer = standard_answer[question]
    scores = []
    for submitted_answer in answers:
        submitted_answer = str(submitted_answer)
        if submitted_answer.strip() == "":
            scores.append(0)
            continue
        score = similarity_score(answer, submitted_answer)
        scores.append(score)
    return scores

In [ ]:
import os
import pandas as pd


def get_the_list_of_files(path):
    """
    Get the list of files in the directory
    """
    files = []
    for dirpath, dirnames, filenames in os.walk(path):
        files.extend(filenames)
        break
    return sorted(files)


images = get_the_list_of_files(base_path_images)

# get max page from annotations_list
max_page = 0
for annotation in annotations_list:
    if annotation["page"] > max_page:
        max_page = annotation["page"]
max_page = max_page + (1 if max_page % 2 == 1 else max_page + 2) # Scanner will have a blank page!

# filter images by file name divided by page
images_by_page = []
for page in range(max_page):
    images_by_page.append([])
    for image in images:
        p = int(image.split(".")[0])
        if p % max_page == page:
            images_by_page[page].append(image)


def get_df(question):
    row = annotations_dict[question].copy()

    row["Confidence"] = 0.1
    row["Similarity"] = 0
    row["Image"] = images_by_page[row["page"]]
    # append base_path_images to each image
    row["Image"] = ["images/" + image for image in row["Image"]]

    # expend row to dataframe for each image in row["Image"]
    data = pd.DataFrame(row)
    data = data.explode("Image")
    data = data.reset_index(drop=True)

    data["Answer"] = data.apply(
        lambda row: ocr_image_from_file(question,
            base_path + "/" + row["Image"],
            row["left"],
            row["top"],
            row["width"],
            row["height"],
        ),
        axis=1,
    )
    # add column RowNumber
    data["RowNumber"] = data.index + 1
    data["maskPage"] = data["page"]

    similarties = calculate_similarity(data["Answer"].tolist(), question)

    data["Similarity"] = similarties

    data["page"] = data["Image"].apply(
        lambda x: x.replace("images/", "").replace(".jpg", "")
    )
    data["Mark"] = data["Answer"].apply(lambda x: "0" if len(x.strip()) == 0 else "")

    return data


def save_template_output(output, question, filename):
    path = Path(base_path_questions, question)
    path.mkdir(parents=True, exist_ok=True)
    path = Path(os.path.join(path, filename))
    text_file = open(path, "w")
    text_file.write(output)
    text_file.close()


# question = "NAME"
# get_df(question)

Generate individual question page.

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

max_count = len(questions)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

# for question in questions:
#     dataTable = get_df(question)
#     os.makedirs(base_path_questions + "/" + question, exist_ok=True)
#     dataTable.to_csv(base_path_questions + "/" + question + "/data.csv", index=False)

#     if question == "ID" or question == "NAME" or question == "CLASS":
#         template = env.get_template("questions/index-answer.html")
#     else:
#         template = env.get_template("questions/index.html")
#     output = template.render(
#         studentsScriptFileName=file_name,
#         question=question,
#         standardAnswer=standard_answer[question] if question in standard_answer else "",
#         standardMark=standard_mark[question] if question in standard_mark else "",
#         estimatedBoundingBox=annotations_dict[question],
#         dataTable=dataTable,
#     )
#     save_template_output(output, question, "index.html")

#     template = env.get_template("questions/question.js")
#     output = template.render(
#         dataTable=dataTable,
#         estimatedBoundingBox=annotations_dict[question],
#     )
#     save_template_output(output, question, "question.js")

#     template = env.get_template("questions/style.css")
#     output = template.render(
#         dataTable=dataTable,
#     )
#     save_template_output(output, question, "style.css")
#     f.value += 1

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import pandas as pd

max_count = len(questions)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

for question in questions:
    # dataTable = get_df(question)
    # os.makedirs(base_path_questions + "/" + question, exist_ok=True)
    # dataTable.to_csv(base_path_questions + "/" + question + "/data.csv", index=False)

    data_path = base_path_questions + "/" + question + "/data.csv"
    dataTable = pd.read_csv(data_path)
    dataTable = dataTable.replace(".*No text found!!!.*", "", regex=True)
    similarties = calculate_similarity(dataTable["Answer"].tolist(), question)
    dataTable["Similarity"] = similarties
    dataTable.to_csv(base_path_questions + "/" + question + "/data.csv", index=False)

    if question == "ID" or question == "NAME" or question == "CLASS":
        template = env.get_template("questions/index-answer.html")
    else:
        template = env.get_template("questions/index.html")
    output = template.render(
        studentsScriptFileName=file_name,
        question=question,
        standardAnswer=standard_answer[question] if question in standard_answer else "",
        standardMark=standard_mark[question] if question in standard_mark else "",
        estimatedBoundingBox=annotations_dict[question],
        dataTable=dataTable,
    )
    save_template_output(output, question, "index.html")

    template = env.get_template("questions/question.js")
    output = template.render(
        dataTable=dataTable,
        estimatedBoundingBox=annotations_dict[question],
    )
    save_template_output(output, question, "question.js")

    template = env.get_template("questions/style.css")
    output = template.render(
        dataTable=dataTable,
    )
    save_template_output(output, question, "style.css")
    f.value += 1

## Validate Student ID

In [ ]:
# load csv file to dataframe
import pandas as pd

id_from_oscr = pd.read_csv(base_path_questions + "/" + "ID" + "/data.csv")["Answer"].tolist()
id_from_oscr = [str(int(float(x))) if pd.notna(x) else x for x in id_from_oscr]

id_from_namelist = name_list_df["StudentID"].to_list()

# check duplicate id
duplicate_id = []
for id in id_from_oscr:
    if id_from_oscr.count(id) > 1:
        duplicate_id.append(id)
duplicate_id = list(set(duplicate_id))
if len(duplicate_id) > 0:
    print(colored("Duplicate ID: {}".format(duplicate_id), "red"))

id_from_oscr = [str(id) for id in id_from_oscr]
id_from_namelist = [str(id) for id in id_from_namelist]

# compare oscr_id and validate_id
ocr_missing_id = []
name_list_missing_id = []
for id in id_from_oscr:    
    if id not in id_from_namelist:       
        name_list_missing_id.append(id)

for id in id_from_namelist:
    if id not in id_from_oscr:   
        ocr_missing_id.append(id)

## OCR scan error case

In [ ]:
from termcolor import colored
if len(ocr_missing_id) > 0:
    print(colored("Some IDs OCR is not in NameList and you need to fix it manually!", "red"))
    for id in name_list_missing_id:
        print(colored(id, "red"))

## Potential Absent Case

In [ ]:
from termcolor import colored

if len(ocr_missing_id) > 0:
    print(colored("Number of absentee {}.".format(len(ocr_missing_id)), "red"))
    print(colored("ID in Name List does not find from OCR!", "red"))
    for id in ocr_missing_id:
        print(colored(id, "red"))

# Start Python HTTPServer

The webserver log is in output/server.log.

If you are in development and don't want the notebook being blocked by running webserver, you can open a terminal and run the below command.

file_name=XXXX python server.py 8000

In [ ]:
print("file_name={} python server.py".format(file_name))

In [ ]:
# You can also uncomment the following line to run the web server but if it crashes, you need to restart the kernel.
# !cd .. && file_name=TestScript python server.py

# Post Processing after scoring
1. Check all question has scores.
2. Check ID again.
3. Remove version history.

In [ ]:
# check each sub folder of base_path_questions contains file name mark.json, ignore the root folder
import os
import json

unfinsihed_scoring = []
for path, currentDirectory, files in os.walk(base_path_questions):
    if path != base_path_questions:
        # extract question name from path
        question = path[len(base_path_questions) + 1 :]
        if "mark.json" not in files:
            unfinsihed_scoring.append(question)
        else:
            # read mark.json as json
            with open(os.path.join(path, "mark.json"), "r") as f:
                marks = json.load(f)            
            # check each mark in marks that attribute "mark" or "overridedMark" is not empty
            for mark in marks:
                if mark['mark'] == "" and  mark['overridedMark'] == "":
                    # extract question name from path                   
                    unfinsihed_scoring.append(question)
                    break             

if len(unfinsihed_scoring) > 0:            
    print(colored("{} have some question not yet mark!".format(unfinsihed_scoring), "red"))          
else:
    print("All questions have been marked!")

Check ID

In [ ]:
import os
import json
from termcolor import colored

with open(os.path.join(base_path_questions,"ID", "mark.json"), "r") as f:
    marks = json.load(f)

id_from_mark = list(map(lambda x: x["overridedMark"] if x["overridedMark"] != "" else x["mark"], marks))
id_from_namelist = name_list_df["StudentID"].to_list()

# convert id_from_mark to string
id_from_mark = [str(id) for id in id_from_mark]
id_from_namelist = [str(id) for id in id_from_namelist]

mark_missing_id = []
for id in id_from_namelist:
    if id not in id_from_mark:   
        mark_missing_id.append(id)
print(colored("In class but not marked - {}!".format(mark_missing_id), "red"))    

marked_but_not_in_namelist = []
for id in id_from_mark:
    if id not in id_from_namelist:   
        marked_but_not_in_namelist.append(id)

print(colored("Marked ID but not in class - {}!".format(marked_but_not_in_namelist), "red"))

### Remove version history
Before you backup.

In [ ]:
## remove fill start with control- or mark- and end with .json in base_path_questions recursively.
import os
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file.startswith("control-") or file.startswith("mark-"):
            os.remove(os.path.join(path, file))

### Reset everything (Danger)
Remove mark.js and control.js

In [ ]:
# import os
# for path, currentDirectory, files in os.walk(base_path_questions):
    
#     for file in files:       
#         if file == "control.json" or file == "mark.json":
#             os.remove(os.path.join(path, file))